In [1]:
import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

INFO 04-22 13:46:33 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
%env CUDA_VISIBLE_DEVICES=2

In [ ]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Tianyi-Lab/Personas")
personas = ds['train']['Llama-3.1-70B-Instruct_descriptive_persona']

In [3]:
np.random.seed(seed)
random_indices = np.random.choice(len(personas), size=200, replace=False).astype(int)
personas_updated = np.array(personas)[random_indices]

In [4]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [5]:
import json
import shutil

# Step 1: Read existing config
with open('config/chit-chat/prompts.json', 'r') as f:
    config_prompt = json.load(f)

# Step 2: Overwrite values
config_prompt['dialogue_prompt'] = "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is meeting the other for the first time hoping to get to know the other better. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY% Your conversation so far is below:\nConversation: %CONVERSATION%"
config_prompt['reminder_prompt'] = "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: "

config_prompt['eval_prompts']['prompt_consistency'] =  "For the following line spoken by %SPEAKER_ROLE%, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%. Note that it is okay for %SPEAKER_ROLE% to provide details not in their background, but these details cannot be inconsistent with the background. %SPEAKER_ROLE%'s background is: %SPEAKER_BACKSTORY% %SPEAKER_ROLE% spoke the following line: %SPEAKER_LINE% Answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%, followed by 1 sentence of reasoning. Be very strict in your assessment but remain accurate."

# Step 3: Backup
with open('config/chit-chat/prompts.json', 'w') as f:
    json.dump(config_prompt, f, indent=4)

In [6]:
config_prompt

{'agent1_role': 'Person A',
 'agent2_role': 'Person B',
 'dialogue_prompt': 'You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is meeting the other for the first time hoping to get to know the other better. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY% Your conversation so far is below:\nConversation: %CONVERSATION%',
 'eval_prompts': {'prompt_consistency': "For the following line spoken by %SPEAKER_ROLE%, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%. Note that it is okay for %SPEAKER_ROLE% to provide details not in their background, but these details cannot be inconsistent with the background. %SPEAKER_ROLE%'s background is: %SPEAKER_BACKSTORY% %SPEAKER_ROLE% spoke the following line: %SPEAKER_LINE% Answer YES if t

In [8]:
config_prompt['eval_prompts']['prompt_consistency']

"For the following line spoken by %SPEAKER_ROLE%, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%. Note that it is okay for %SPEAKER_ROLE% to provide details not in their background, but these details cannot be inconsistent with the background. %SPEAKER_ROLE%'s background is: %SPEAKER_BACKSTORY% %SPEAKER_ROLE% spoke the following line: %SPEAKER_LINE% Answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%, followed by 1 sentence of reasoning. Be very strict in your assessment but remain accurate."

In [9]:
# personas_new = []

# for persona in personas_updated:
#     name_prompt = "Given the following biography below, parse the name of the person. \nOnly output the name.\n" + persona + "\nName: "
#     name = completion_create(config['agent1_model'], config, name_prompt)
#     name = get_text_after_colon("Name", name)
#     print(name)    
#     personas_new.append({'persona': persona, 'name': name})

# with open('config/chit-chat/personas.json', 'w') as f:
#     json.dump(personas_new, f, indent=4)

In [10]:
with open("config/chit-chat/personas.json", "r") as f:
    personas_updated = json.load(f)

In [ ]:
# choose config file to load into global config dictionary in utils.py
with open("./config/chit-chat/Llama-3.1-8B-Instruct.json", 'r') as f:
    config_llm = json.load(f)

config_llm["eval_model"] = "gpt4o-mini"

for key, value in config.items():
    config_llm[key] = value

# this modifies the global prompts dictionary in utils.py
with open('config/chit-chat/prompts.json', 'r') as f:
    prompts = json.load(f)
    
import os
from datetime import datetime
import chit_chat_generation 
# Create timestamped directory
current_date = str(datetime.now().strftime("%m.%d.%y"))

output_dir = f"data/chit-chat/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)
# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_1.json")
chit_chat_generation.config = config_llm
chit_chat_generation.config = config_llm
chit_chat_generation.prompts = prompts

utils.config = config_llm
index_offset = load_stats_file(write_file)

conversations = []
for p_dict1, p_dict2 in tqdm(np.array(personas_updated[:20]).reshape(-1, 2)):
    chit_chat_generation.reset_stats(config_llm)
    conversation = chit_chat_generation.generate_conversation(
        config_llm,
        p_dict1["persona"], 
        p_dict2["persona"], 
        p_dict1["name"], 
        p_dict2["name"], 
        pturn=1
    )
    print(conversation)
    conversations.append(conversation)
    stats['index'] = index_offset
    stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    write_stats(write_file)
    index_offset += 1

with open(write_file, 'r') as f:
    conversations = json.load(f)
    


write_stats(write_file)

  0%|                                                                                                                 | 0/10 [00:00<?, ?it/s]

Llama-3.1-8B-Instruct
INFO 04-21 15:35:28 [config.py:600] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 04-21 15:35:28 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-21 15:35:30 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/home/ryan_cheng/models/', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observabil

In [ ]:
config_prompt

In [17]:
import consistency_eval

path_convos = "data/chit-chat/exp/04.21.25/Llama-3.1-8B-Instruct_0_1.json"
with open(path_convos, 'r') as f:
    path_convos_json = json.load(f)
    
consistency_eval.prompts = config_prompt


for conversation in tqdm(path_convos_json):
    consistency_eval.eval_prompt_consistency(conversation)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 9098.27it/s]


In [18]:
path_convos_json

[{'task_name': 'Anthology',
  'P1': '"Meet 49-year-old Alexandra \\"Alex\\" Thompson, a vibrant and determined woman living in Charleston, South Carolina. Born and raised in the heart of the Lowcountry, Alex embodies the warmth and hospitality that her home state is famous for. As a white woman, she has had her share of privileges, but she\'s also faced challenges that have shaped her into the strong, independent individual she is today.\\n\\nAlex grew up in a traditional Southern family, where her parents valued hard work, education, and community involvement. Her father was a high school history teacher, and her mother worked as a librarian, instilling in Alex a love for learning and a strong sense of social responsibility. She has two younger brothers, both of whom have moved away from South Carolina to pursue their careers.\\n\\nAfter graduating from the University of South Carolina with a degree in environmental science, Alex worked for several years as a conservation biologist fo